# Welcome to 画像生成 AI tutorial ‼

**対象レベル**  
このチュートリアルはPythonチュートリアルの [4. その他の制御フローツール](https://docs.python.org/ja/3/tutorial/controlflow.html)の関数定義までの知識があることを前提としています。

このtutorialではGAN(敵対的生成ネットワーク)を用いた画像生成についてやっていきます。コードブロックを一つ一つ実行していくと画像を生成してくれるAIモデルが出来上がります。  
ここに出てくるプログラムコードを自分の手で書き写して実行してみると良いと思います。 
ベースとなっているより詳しいチュートリアルはこちらです。  
日本語: [【PyTorchチュートリアル⑪】DCGAN Tutorial](https://qiita.com/sudominoru/items/02f4b6313585f14b210c)  
英語:[DCGAN TUTORIAL](https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html)  
それでは始めていきましょう！

### 目次

### 注意事項
このチュートリアルは演算負荷が高いためGPU環境が必要です。  
Google Colab で実行している方は、ページ上部から**ランタイム** &rarr; **ランタイムのタイプを変更** をクリックし **ハードウェアアクセラレータ** を *None* から *GPU* に変更してください。  
GPUが使用可能かどうかは次のコードブロックを実行することで分かります。

In [ ]:
import torch
print("GPU:",torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using:",device)

# ・下準備
さて、AIを作るための下準備をしていきましょう。勝手に画像を生成するAIが作れたらうれしいのですが、残念ながらそう簡単にはいきません。  
AIを作るための先人の知恵と努力の塊である「ライブラリ」を使いますし、学習するためには大きなデータセットが必要です。  
それではサクッと下準備をしてしまいましょう。

### ライブラリ
```import Namae```や```from Namae1 import Namae2```ですでにインストールされているライブラリを使います。これからもちょくちょく出てきます。  
ちなみにPythonのライブラリの主なインストール方法は```pip install Namae```です。詳しくは[公式ドキュメント](https://docs.python.org/ja/3/installing/index.html)を参照ください。  

In [ ]:
import torch # Deep Learningのためのライブラリです。
import torchvision # 画像処理のためのライブラリです。
import torchvision.transforms as transforms # 画像処理のライブラリから「変換」に特化したものを持ってきます
import numpy as np # 行列計算のためのライブラリです。

私がよく使っている```torchsummaryX```をインストールします。モデルのパラメータ数などを見ることができるため重宝しています。

In [ ]:
%pip install torchsummaryX

### データセット
画像を生成することにもデータセットが必要です。画像生成といってもまったく新しい画像を一から生成するわけではなく、**あくまでデータセットに似ている新しい画像**を生成します。データセットがバリエーション豊かで膨大であればそれだけたくさんの種類の画像を生成することができます。  
今回は犬の猫の画像のデータセットを使っていきましょう。(https://www.kaggle.com/zippyz/cats-and-dogs-breeds-classification-oxford-dataset)


データセットをダウンロードします。

データセットを前処理します。  
多くの場合、AIモデルの入力は決まった形のデータである必要があります。ただの画像データを```torch.Tensor```という型にしたり、今回の場合は64x64にリサイズしたりします。
今回使うImageFolderクラスの仕様上、サブフォルダーから読み込むので画像があるフォルダの一つ上を指定します。  
```
dataroot = path/to/images
path/to/images
    -> dog_cat_images 
        -> cat.jpg
        -> dog.jpg
        ...
```


In [ ]:
from torchvision.datasets import ImageFolder # 画像があるフォルダをデータセットとして読み込むために使います。

dataroot = "data/images" # 仕様上、サブフォルダーから読み込むので画像があるフォルダの一つ上を指定します。
image_size = 64 # すべての画像はResizeとCentorCropを使用してこのサイズに変更されます。
data_trans = transforms.Compose([       # Composeは処理をまとめるという意味です。
    transforms.Resize(image_size),      # Resizeするときの辺の長さです。
    transforms.CenterCrop(image_size),  # 画像の縦横の長さが異なる場合は中央を切り取ります。
    transforms.ToTensor(),              # 画像データをtorch.Tensor型にします。ついでに各ピクセルの値が
                                        # 0~1に正規化されます。
    transforms.Normalize(mean=0.5, std=0.5),    # 各ピクセルの値からmeanを引き、stdで割ることで
                                                # -1~1の範囲に正規化します。
])
# datarootとdata_trans(前処理)をImageFolderに渡してデータセットは完成です。
dataset = ImageFolder(root=dataroot,transform=data_trans)

データローダーを定義し、どんなデータセットか見てみましょう。

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt # 可視化のためのグラフ描画ライブラリです。

BATCH_SIZE = 128 # 一度に取り出すデータの個数。大きいほど学習が速く進むが、計算量やメモリ使用料が増える。

dataloader = torch.utils.data.DataLoader( # データの取り出し方を決めるものです。
    dataset,batch_size=BATCH_SIZE,
    shuffle=True # 全データをシャッフルしながら取り出します。（重複なし）
)

real_batch = next(iter(dataloader)) # 1バッチ分取り出して
real_batch = real_batch[0] # (data, )というタプル構造になっているのでこの処理が入ります。
images = torchvision.utils.make_grid(real_batch[:64],padding=True, normalize=True) # 64枚をひとまとめにした画像を作って
images = np.transpose(images.numpy(),(1,2,0)) # 描画用のデータ形式にして
plt.figure(figsize=(8,8)) # 8x8inchの空の図を作成し
plt.axis("off") # 軸の描画を切ります。
plt.title("Training Images") # タイトルをつけて
plt.imshow(images) # 画像を埋め込み、
plt.show() # 描画！


# ・AIモデルの定義
さあAIモデルを定義していきましょう。GANが発表された当時は非常にトリッキーで注目されました。  
GANは敵対的生成ネットワークと訳されるように、Generator(生成器)とDiscriminator(判別器)が競い合うことによって学習します。DiscriminatorはGeneratorが生成した画像か本物の画像が分類できるように学習します。GeneratorはDiscriminatorを騙せる本物のような画像を生成するように学習します。 
例えるなら、偽札を作る犯罪者と取り締まる警察でしょうか。このようなループを繰り返すことによってGeneratorが成長し、画像が生成されていくのです！  

### Generator (生成器)
まずはGeneratorを定義していきましょう。すこし理解するのが難しいかもしれませんが、Generatorの入力には標準分布に従うランダムノイズを投入します。ランダムなノイズから画像を生成すると考えればよいでしょう。

In [ ]:
import torch.nn as nn # ある一つの機能を持った、「レイヤー」というものをひとまとめにしたものです。

nc = 3 # トレーニング画像のチャネル数。カラー画像の場合は「3」 
nz = 100 # Generatorに入力するランダムノイズのサイズです。
ngf = 64 # num generator features の略で、モデルの大きさを設定するハイパーパラメータにします。

class Generator(nn.Module): # nn.Moduleを継承して、
    def __init__(self):     # コンストラクタを定義します。
        super().__init__()  # 継承元のコンストラクタを呼び出します。

        self.main = nn.Sequential( # この中にレイヤーを順々に定義していきます。定義した順にしたがってデータは通されます。
            # 入力は BATCH_SIZE x nz x 1 x 1 のサイズのランダムノイズです。
            nn.ConvTranspose2d( # 画像を拡大する畳み込み層です。 
                in_channels=nz, out_channels=ngf*8,# 画像のカラーチャネルをnz -> nfg*8 にします。
                kernel_size=(4,4), # 畳み込みレイヤーのフィルターサイズです。
                stride=1,   # フィルターを横にスライドする時のステップするピクセル数です。出力される画像サイズは約stride倍されます。
                padding=0,  # 画像の縁を何ピクセル増やすかを指定します。出力されるデータの形に影響します。
                bias=False  # バイアスという加算のみをするパラメータをつけるか否かです。 
            ),
            nn.BatchNorm2d(num_features=ngf * 8), # データの各チャネルをバッチ事に平均0、分散１に正規化します。
            nn.ReLU(inplace=True), # Rectified Linear Unit という関数に通します。
            # サイズ (ngf*8) x 4 x 4 （この時点で）
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False), # 先ほどの畳み込み層の定義と同様です。
            nn.BatchNorm2d(ngf * 4), # 先ほどの正規化する層と同様です。
            nn.ReLU(True),
            # サイズ  (ngf*4) x 8 x 8（この時点で）
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2), 
            nn.ReLU(True),
            # サイズ (ngf*2) x 16 x 16（この時点で）
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # サイズ (ngf) x 32 x 32（この時点で）
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh() # 出力するデータの値域を -1 ~ 1 にするために使います。
            # サイズ (nc) x 64 x 64（この時点で）
        )

    def forward(self, input): # forwardメソッドを定義して、
        return self.main(input) # 先ほど定義したSequentialにinputデータを通して、値を返します。

### Discriminator (判別器)
Generatorの定義が終わったので、今度は実際のデータかGeneratorが生成したデータかを見分ける Discriminatorを定義していきましょう。Discriminatorの入力は3x64x64の画像で、出力は真偽を表す 1つだけです。

In [ ]:
#nc = 3 # ncは先ほど定義してあるためここでは定義しません。
ndf = 64 # num discriminator features の略で、モデルの大きさを設定するハイパーパラメータにします。
Leaky_slope = 0.2 # LeakyReLUの負の傾斜の値です。

class Discriminator(nn.Module): # nn.Moduleを継承して、
    def __init__(self):         # コンストラクタを定義します。
        super().__init__()      # 継承元のコンストラクタを呼び出します。
        
        self.main = nn.Sequential( # Generatorの定義時と同様です。
            # 入力は (nc) x 64 x 64
            nn.Conv2d( # 畳み込みレイヤーです。
                in_channels=nc, out_channels=ndf, # 画像のチャネルを nc -> ndfにします。
                kernel_size=4,  # 先ほどと同様に畳み込みレイヤーのフィルターサイズです。
                stride=2,       # 画像サイズはおおよそ 1/strideになります。
                padding=1,      # 入力画像に上下左右に1ピクセル 0 を追加します。
                bias=False      # バイアスという加算のみをするパラメータをつけるか否かです。  
            ),
            nn.LeakyReLU(negative_slope=Leaky_slope, inplace=True), # Leaky Rectified Linear Unitという関数に通します。
            # サイズ (ndf) x 32 x 32 （この時点で）
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False), # 先ほどの畳み込み層の定義と同様です。
            nn.BatchNorm2d(ndf * 2), # 先ほどと同じ正規化する層です。
            nn.LeakyReLU(0.2, True), 
            # サイズ (ndf*2) x 16 x 16（この時点で）
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, True),
            # サイズ (ndf*4) x 8 x 8（この時点で）
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, True),
            # サイズ (ndf*8) x 4 x 4（この時点で）
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid() # データの値域を 0 ~ 1 にします。
        )

    def forward(self, input): # forwardメソッドを定義して
        return self.main(input) # 先ほど定義したSequentialに入力データを通します。

### 各レイヤーについて
このチュートリアルでは各レイヤーの仕組みについては解説しませんので、気になる方は次のリンクを参照ください。  
[Conv2d]()  
[ConvTranspose2d]()  
[BatchNorm2d]()  
[ReLU]()  
[LeakyReLU]()  
[Tanh]()  
[Sigmoid]()  

### 重み初期化について
DCGANの重みの値はすべて平均0.0, 標準偏差 0.02に従ったガウス分布によって初期化されます。（論文中で詳しくその理由は明かされませんでした）現在はもっと良い重み初期化の手法が確立されていると思います。

In [ ]:
# G（生成器）とD（識別器）の重みの初期化
def weights_init(m): # `m`odule（レイヤー）を受け取って
    classname = m.__class__.__name__    # 名前の中に
    if classname.find('Conv') != -1:    # `Conv`が含まれていれば
        nn.init.normal_(m.weight.data, 0.0, 0.02) # 平均0.0, 標準偏差0.02の標準(normal)分布で初期化
    elif classname.find('BatchNorm') != -1: # `BatchNorm`が含まれていれば
        nn.init.normal_(m.weight.data, 1.0, 0.02) # 平均1.0, 標準偏差0.02で初期化します。
                                                  # BatchNormのパラメータは少々特殊で、乗数γとして扱われるので平均1.0になります。
        nn.init.constant_(m.bias.data, 0)   # 加算のみが行われるバイアス項の初期値は0です。

### モデルのインスタンス化 (コンストラクト)
さて、先ほどまでで定義したモデルをインスタンス化します（設計図から実際のものを作るようなものです。）  
Pytorchで作ったAIモデルはデータを通すまで実はモデルの形（計算グラフ）が構築されません。ですのでダミーデータを作成してモデル通し、実際のデータの流れや計算量を可視化します。


In [ ]:
from torch.utils.tensorboard import SummaryWriter # `tensorboard`というグラフィカルな可視化ツールに記録するためのものです。
from torchsummaryX import summary # 計算量や学習可能パラメータの数等を見るためのツールです。

# 実行するデバイスを決定する
if torch.cuda.is_available(): # GPU (cuda) が使えるのなら
    device = "cuda:0" # deviceに"cuda:0" を代入
else:                 # 使えなければ
    device = "cpu"    # CPU を使う

device = torch.device(device) # torch.deviceクラスに変換します。この行がなくても動きます。

netG = Generator().to(device) # Generatorを構築し、演算デバイスへ送ります。
netD = Discriminator().to(device) # Discriminatorを構築し、演算デバイスへ送ります。

netG.apply(weights_init) # 重み初期化を適用します。内部でレイヤーが一つ一つ
netD.apply(weights_init) # weight_init(m)に通されていきます。

### モデルの可視化
ダミーデータを使ってモデルを可視化します。Tensorboard記録されたモデルの形（計算グラフ）を見るための方法は後述の **記録について** で説明しますのでそのまま実行してください。

In [ ]:
dummy_g_input = torch.randn(1,nz,1,1, device=device) # 1 x 100 x 1 x 1 のダミーデータを用意して
dummy_d_input = torch.randn(1,nc,image_size,image_size,device=device) # 1 x 3 x 64 x 64 のダミーデータを用意して

print("Generator")          # Generatorの計算量やパラメータ数を出します。
summary(netG,dummy_g_input) # 勝手に内部でGeneratorにダミーデータを通してくれます。
print("Discriminator")      # Discriminatorも同様に
summary(netD,dummy_d_input) # します。

In [ ]:
writer = SummaryWriter(log_dir="runs/Generator") # Generator用のログを作ります。
writer.add_graph(netG,dummy_g_input) # 勝手に内部でGeneratorにダミーデータを通してくれます。
writer.close()  # ちゃんと閉じて二回連続で書き込まれないようにします。

In [ ]:
writer = SummaryWriter(log_dir="runs/Discriminator") # 先ほどと同様にDiscriminator用のログディレクトリを作って
writer.add_graph(netD, dummy_d_input) # 通して
writer.close() # 閉めます。

# ・学習

さあモデルの定義も可視化も終わったことですし、ついに学習の流れを決めてきます。 GANはGeneratorとDiscriminatorを交互に学習するため少し手順が複雑です。さらに、GeneratorとDiscriminatorを競わせるため、`誤差` の取り方がそれぞれ異なります。  
そもそも、深層学習のモデルは、出力と答えの **損失（誤差）** を最小化しようと学習（重みパラメータの更新）が行われます。
今回使う `BCELoss` (Binary Cross Entropy Loss) は、出力を1か0のどちらか一方に集中させたいときに使います。正解ラベルを$y \ | \ 0 \ or \ 1$, 出力を$x \ | [0,1]$ (0から1の範囲)とすると次のように損失（誤差）は定義されます。
$$ -(y ⋅log \ x +(1−y)⋅log(1−x))$$
対数$-log \ x$は、$x$ が1の時が最小になり、$-log(1-x)$ はxが0のときに最小になります。  
つまり、正解ラベル$y$が 1 のときは上の式は$-log \ x $ となり$x$は1になるように学習されますし、  
正解ラベル$y$が 0 のときは上の式は $-log(1-x)$ となり$x$は 0 になるように学習されます。  

In [ ]:
from torch.optim import Adam # 重みの更新手法はAdamを採用します。

# BCELoss関数を初期化します
criterion = nn.BCELoss()

# ジェネレータの進行を視覚化するために使用する潜在ベクトルを作成します
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# トレーニング中に本物のラベルと偽のラベルのルールを設定します
real_label = 1. # True
fake_label = 0. # False

# G と D に Adam オプティマイザを設定する
lr = 0.0002 # 大きいほど1ステップでモデルの重みが大きく更新されます。論文中では 0.001が大きすぎてこの値だと安定したそうです。
beta1 = 0.5 # 普段は0.9くらいなのですが、学習が不安定になるようなのでこの値となります。
optimizerD = Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999)) # Discriminatorを更新するoptimizerを用意します。
optimizerG = Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999)) # パラメータと学習率の設定は必須ですが、betasはデフォルト値を使うことも多いです。

そして実際の学習フローでは、最初にDiscriminatorを更新し、次にGeneratorを更新するという流れを繰り返していきます。  
Discriminatorは実際のデータに対しては 1 を出力するように学習し、Generatorのフェイク画像に対しては 0 を出力するように学習します。  
Generatorは画像を生成し、それをDiscriminatorに通した結果が 1 になるように学習されます。

In [2]:
from datetime import datetime # 日付を取得するためのライブラリです。
num_epochs = 5 # 全データを使って重みを更新する回数です。世代数と呼ばれたりもします。
log_interval = 10 # 記録するときのインターバルステップ数です。

def train(): # 学習関数を定義して、
    print("Start Training roop...") # 見えるようにプリント
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S") # 日付を取ってきて文字列にして、
    writer = SummaryWriter(log_dir="runs/"+now) # 記録writerを初期化します。
    current_step = 0 # 現在のステップ数
    img_list = []   # 1 epoch ごとにGeneratorの生成した画像をここに入れます。あとで見るために。

    for epoch in range(num_epochs): # 世代数のループを定義して、
        
        for i, data in enumerate(dataloader): # BATCHとなるdataとそのインデックス i をまわして、 
            ### Discirminator の更新 #################
            ## 真の画像でトレーニングします。
            netD.zero_grad() # 重み更新のための勾配を0で初期化します。

            real_img = data[0].to(device) # データを演算デバイスに送って、
            b_size = real_img.size(0)   # batch_sizeを取得して、（最後のバッチはバッチサイズに満たないこともあるので）
            label = torch.full((b_size, ),fill_value=real_label) # 1 で埋めたサイズ b_size のテンソルを作り、
            label = label.type_as(real_img) # real_imgと同じデバイス、数値型にします。

            output = netD(real_img) # Discriminatorに現実の画像を通して、
            output = output.view(-1) # 形が(b_size x 1)なので (b_size )にして
            errD_real = criterion(output, label) # 誤差を計算します。
            # 逆伝播でDの勾配を計算します
            errD_real.backward() # 誤差から勾配を計算します。
            D_x = output.mean().item() # Discriminatorの現実の画像に対する平均判定率を出します。

            ## 偽の画像でトレーニングします
            noise = torch.randn(b_size, nz, 1, 1, device=device) # 潜在ベクトルのバッチを生成します
            # Gで偽の画像を生成します
            fake = netG(noise) # Generatorに通して、
            label.fill_(fake_label) # ラベルを0で埋めなおして
            # 生成した偽画像をDで分類します
            output = netD(fake.detach()).view(-1)
            # Dの損失を計算します
            errD_fake = criterion(output, label)
            # 勾配を計算します
            errD_fake.backward()
            D_G_z1 = output.mean().item()

            # 実在の画像の勾配と偽画像の勾配（損失）を加算します
            errD = errD_real + errD_fake

            optimizerD.step() # Discriminatorの重みを更新します。

            ### Generator の更新 #################

            netG.zero_grad() # 重み更新のための勾配を0で初期化します。
            label.fill_(real_label)  # 偽のラベルは生成器の損失にとって本物です
            # パラメータ更新後のDを利用して、偽画像を通します
            output = netD(fake).view(-1)
            # この出力に基づいてGの損失を計算します
            errG = criterion(output, label)
            # Gの勾配を計算します
            errG.backward()
            D_G_z2 = output.mean().item()

            optimizerG.step() # Generatorの重みを更新します。

            if current_step % log_interval == 0: # 現在のステップ数がlog_intervalの倍数の時、
                # 結果を出力します。
                print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                    % (epoch, num_epochs, i, len(dataloader),
                    errD.item(), errG.item(), D_x, D_G_z1, D_G_z2)
                )

                # tensorboardにそれぞれLoss D, Loss G を記録します
                writer.add_scalar("Loss_D",errD.item(),current_step)
                writer.add_scalar("Loss_G",errG.item(),current_step)

            current_step += 1 # 次のステップへ
        
        # 1 エポックが終了したら、
        with torch.no_grad(): # 重み更新のための勾配計算を無効化して、
            fake = netG(fixed_noise) # Generatorに画像を生成させます。
            fake = fake.detach().cpu() # 演算デバイスからcpu上に移動させて、
        img = torchvision.utils.make_grid(fake, padding=2, normalize=True) # グリッド画像を作ります。
        img_list.append(img) # img_listに追加して、
        writer.add_image("Generated Image",img,current_step) # Tensorboardにも書き込んで、

        # 1 エポックの処理は終わりです。
    
    # 学習が完了したら、
    writer.close() # 記録Writerを閉じて、

    return img_list # 学習中に保存しておいた img_listを返して終わりです。

##### 実は大変なGANの学習 -モード崩壊について-
もしGeneratorの学習が変な方向に進んだ結果、とてつもなく高品質な画像1枚だけを生成するようになってしまったとしても、Generatorの学習目的としては成功したことになります。こういった生成データのバリエーションが極端に乏しく学習が進んでしまうことを**モード崩壊**といいます。

### 記録について
損失や各エポックでの生成画像ははtensorboardに記録します。tensorboardを起動するためには、ターミナルからこのノートブックのフォルダ内で次のコマンドを実行してください。    
```tensorboard --logdir="runs"```
Google Colabで実行している場合は次のコードブロックを実行してください。


In [ ]:
%load_ext tensorboard
%tensorboard --logdir="runs"

# ・実行してみよう！

In [ ]:
img_list = train() # 学習を走らせます。

### モデルを保存する

In [ ]:
torch.save(netD.state_dict(),"Discriminator.pth") # このノートブックと同じフォルダに
torch.save(netG.state_dict(),"Generator.pth")     # ファイルができているはずです。  

#### モデルをロードする
学習済みのパラメータは次のコードブロックのコメントアウトを外し、

In [ ]:
#netD.load_state_dict(torch.load("Discriminator.pth",map_location=device))
#netG.load_state_dict(torch.load("Generator.pth",map_location=device))

# ・結果を見てみよう！

In [ ]:
from IPython.display import HTML
import matplotlib.animation as animation
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

### ランダムに生成

In [ ]:
with torch.no_grad():
    noise = torch.randn(1,nz, 1,1,device=device)
    fake = netG(noise).cpu() * 0.5 + 0.5
    fake = fake.squeeze(0).permute(1,2,0)
plt.imshow(fake)
plt.show()    

# 参照

In [ ]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# 再現性のためにランダムシードを設定する
manualSeed = 999
#manualSeed = random.randint（1、10000）＃新しい結果が必要な場合に使用
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:
dataroot = "data/images"

# データローダーのワーカー数
workers = 0

# トレーニングのバッチサイズ
batch_size = 128

# トレーニング画像の空間サイズ。
# すべての画像はトランスフォーマーを使用してこのサイズに変更されます。
image_size = 64

# トレーニング画像のチャネル数。カラー画像の場合は「3」 
nc = 3

# 潜在ベクトル z のサイズ（つまり、ジェネレータ入力のサイズ）
nz = 100

# 生成器の feature map のサイズ
ngf = 64

# 識別器の feature map のサイズ
ndf = 64

# エポック数
num_epochs = 50

# 学習率
lr = 0.0002

# Adam オプティマイザのBeta1ハイパーパラメータ
beta1 = 0.5

# 使用可能なGPUの数。0の場合、CPUモードで実行されます
ngpu = 1

In [ ]:
# 画像フォルダデータセットは、以下で設定した方法で使用できます。

# データセットを作成する
dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))
# データローダーを作成する
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# 実行するデバイスを決定する
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

# トレーニング画像をプロットする
real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
# G（生成器）とD（識別器）の重みの初期化
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 入力は Z で、畳み込み層に渡されます
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # サイズ (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # サイズ  (ngf*4) x 8 x 8
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # サイズ (ngf*2) x 16 x 16
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # サイズ (ngf) x 32 x 32
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # サイズ (nc) x 64 x 64
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# ジェネレーターを作成します
netG = Generator(ngpu).to(device)

# 必要に応じてGPUを使用します
if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# weights_init関数を適用して、すべての重みを平均「0」、標準偏差「0.02」でランダムに初期化します。
netG.apply(weights_init)

# モデルを出力します
print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            # 入力は (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # サイズ (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # サイズ (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # サイズ (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # サイズ (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Create the Discriminator
# 識別器を作成します
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
# 必要に応じてGPUを使用します
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
# weights_init関数を適用して、すべての重みを平均「0」、標準偏差「0.02」でランダムに初期化します。
netD.apply(weights_init)

# Print the model
# モデルを出力します
print(netD)

In [ ]:
# BCELoss関数を初期化します
criterion = nn.BCELoss()

# ジェネレータの進行を視覚化するために使用する潜在ベクトルを作成します
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# トレーニング中に本物のラベルと偽のラベルのルールを設定します
real_label = 1.
fake_label = 0.

# G と D に Adam オプティマイザを設定する
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
# トレーニングループ

# 進捗状況を追跡するためのリスト
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loop...")
# エポックごとのループ
for epoch in range(num_epochs):
    # データローダーのバッチごとのループ
    for i, data in enumerate(dataloader, 0):

        ############################
        # (1) Dネットワークの更新：log(D(x)) + log(1 - D(G(z))) を最大化します
        ###########################
        ## 実在の画像でトレーニングします
        netD.zero_grad()
        # バッチのフォーマット
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
        # 実在の写真で D の順伝播させます
        output = netD(real_cpu).view(-1)
        # 損失を計算します
        errD_real = criterion(output, label)
        # 逆伝播でDの勾配を計算します
        errD_real.backward()
        D_x = output.mean().item()

        ## 偽の画像でトレーニングします
        # 潜在ベクトルのバッチを生成します
        noise = torch.randn(b_size, nz, 1, 1, device=device)
        # Gで偽の画像を生成します
        fake = netG(noise)
        label.fill_(fake_label)
        # 生成した偽画像をDで分類します
        output = netD(fake.detach()).view(-1)
        # Dの損失を計算します
        errD_fake = criterion(output, label)
        # 勾配を計算します
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # 実在の画像の勾配と偽画像の勾配を加算します
        errD = errD_real + errD_fake
        # Dを更新します
        optimizerD.step()

        ############################
        # (2) Gネットワ​​ークの更新：log(D(G(z))) を最大化します
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # 偽のラベルは生成器の損失にとって本物です
        # パラメータ更新後のDを利用して、偽画像を順伝播させます
        output = netD(fake).view(-1)
        # この出力に基づいてGの損失を計算します
        errG = criterion(output, label)
        # Gの勾配を計算します
        errG.backward()
        D_G_z2 = output.mean().item()
        # Gを更新します
        optimizerG.step()

        # トレーニング統計を出力します
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # 後でプロットするために損失を保存します
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        # fixed_noiseによる G の出力を保存し、生成器の精度を確認します
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
torch.save(netD.state_dict(),"Discriminator.pth")
torch.save(netG.state_dict(),"Generator.pth")

In [ ]:
#%%capture
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())